### 一、使用基于混合高斯模型的背景提取算法，提取前景并显示(显示二值化图像，前景为白色)。

In [7]:
import cv2 as cv

videoFileName = './video/vtest.avi'

cap = cv.VideoCapture(videoFileName)
fgbg = cv.createBackgroundSubtractorMOG2()  # 创建背景提取对象
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3)) # 数学形态学滤波
thresh = 200

# 通过循环读取视频中的每一帧图像
while True:
    ret, frame = cap.read()
    # 没读到当前帧，结束
    if not ret:
        break

    # 通过apply时时更新当前图像，计算出前景对应的掩膜图像
    fgmask = fgbg.apply(frame)
    # 对前景掩膜图像初步进行过滤
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)
    # 开运算，去噪点 获得前景掩膜
    _, fgmask = cv.threshold(fgmask, 30, 0xff, cv.THRESH_BINARY)

    # 显示每一帧图像对应的前景提取结果(前景掩膜)
    cv.imshow('fgmask', fgmask)

    # 每一帧间隔 30ms
    key = cv.waitKey(30)
    # 按下ESC键，退出
    if key == 27:
        break

# 释放资源
cap.release()
# 关闭窗口
cv.destroyAllWindows()

**运行结果**：此处只展示其中的某一帧前景掩膜

<img align="left" width=500 src="./screenshot/3.jpg" />

### 二、基于一，将前景目标进行分割，进一步使用不同颜色矩形框标记，并在命令行窗口中输出每个矩形框的位置和大小

In [3]:
import cv2 as cv
import random

videoFileName = './video/vtest.avi'

cap = cv.VideoCapture(videoFileName)
fgbg = cv.createBackgroundSubtractorMOG2()  # 创建背景提取对象
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
thresh = 200

# 定义颜色列表 5种
colorList = [(0, 0, 0xff), (0, 0xff, 0), (0xff, 0, 0), (0, 0xff, 0xff), (0xff, 0xff, 0)]

def getColor(i):
    '''通过函数索引获取相应颜色'''
    l = len(colorList)
    if i > l - 1:
        i = l % i
    return colorList[i - 1]

def randomColor():
    '''随机颜色函数'''
    # 使用这个，效果不是很好
    # 这里的顺序是bgr不是rgb
    b = random.randint(0, 255)
    g = random.randint(0, 255)
    r = random.randint(0, 255)
    return (b, g, r)

def labelTargets(frame, fgmask, thresh):
    ''' 用于标记目标的函数 '''

    # 对前景图像各个目标进行计算
    # bin, cnts, heir = cv.findContours(fgmask.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
    _, cnts, _ = cv.findContours(fgmask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # 用于统计计数
    count = 0

    # 开始处理每个轮廓
    for c in cnts:
        # 这里的面积是前景轮廓面积
        area = cv.contourArea(c)
        # 获取位置，宽、高
        x, y, w, h = cv.boundingRect(c)

        # 这里要跳过坐标为(0,0) 它是整个图像画面, 还要跳过区域面积小于指定阈值的
        if ((x == 0 and y == 0) or area < thresh):
            continue

        # 获取一种随机颜色，用于标记矩形
        # color = randomColor()
        color = getColor(count)

        count += 1

        # 在当前帧上绘制矩形圈出运动目标
        cv.putText(frame, str(count), (x, y), cv.FONT_HERSHEY_PLAIN, 2, color)
        cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv.drawContours(frame, c, -1, (0, 0, 255), 2) # 绘制轮廓

        # 输出每个矩形框的位置和大小
        print("第{}个矩形框的坐标位置是：({}, {}), 周长是: {}, 面积是: {}".format(count, x, y, 2 * (w + h), w * h), end="  \n")

    if count:
        print('经统计, 共{}个目标'.format(count, end=" \n"))

# 通过循环读取视频中的每一帧图像
while True:
    ret, frame = cap.read()
    # 没读到当前帧，结束
    if not ret:
        break

    # 通过apply时时更新当前图像，计算出前景对应的掩膜图像
    fgmask = fgbg.apply(frame)
    # 对前景掩膜图像初步进行过滤
    fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)  # 开运算，去噪点
    _, fgmask = cv.threshold(fgmask, 30, 0xff, cv.THRESH_BINARY)

    # 得到时时更新后的背景图像
    bgImage = fgbg.getBackgroundImage()

    # 开始标记各个分割目标
    labelTargets(frame, fgmask, thresh)

    # 用于分隔输出
    print()

    # cv.imshow('Background', bgImage)
    cv.imshow('frame', frame)
    # cv.imshow('fgmask', fgmask)  # 显示每一帧图像对应的前景提取结果

    # 每一帧间隔 30ms
    key = cv.waitKey(150)
    # 按下ESC键，退出
    if key == 27:
        break

# 释放资源
cap.release()
# 关闭窗口
cv.destroyAllWindows()


第1个矩形框的坐标位置是：(631, 240), 周长是: 286, 面积是: 4840  
第2个矩形框的坐标位置是：(253, 218), 周长是: 286, 面积是: 4692  
第3个矩形框的坐标位置是：(497, 172), 周长是: 188, 面积是: 1848  
第4个矩形框的坐标位置是：(506, 157), 周长是: 106, 面积是: 682  
经统计, 共4个目标

第1个矩形框的坐标位置是：(631, 241), 周长是: 284, 面积是: 4816  
第2个矩形框的坐标位置是：(257, 219), 周长是: 284, 面积是: 4680  
第3个矩形框的坐标位置是：(496, 157), 周长是: 230, 面积是: 2754  
经统计, 共3个目标

第1个矩形框的坐标位置是：(645, 296), 周长是: 148, 面积是: 1365  
第2个矩形框的坐标位置是：(611, 240), 周长是: 256, 面积是: 3367  
第3个矩形框的坐标位置是：(264, 232), 周长是: 134, 面积是: 912  
第4个矩形框的坐标位置是：(279, 216), 周长是: 232, 面积是: 2635  
第5个矩形框的坐标位置是：(494, 159), 周长是: 230, 面积是: 2844  
经统计, 共5个目标

第1个矩形框的坐标位置是：(592, 241), 周长是: 266, 面积是: 3960  
第2个矩形框的坐标位置是：(284, 214), 周长是: 188, 面积是: 1984  
第3个矩形框的坐标位置是：(491, 161), 周长是: 218, 面积是: 2418  
经统计, 共3个目标

第1个矩形框的坐标位置是：(586, 241), 周长是: 254, 面积是: 3526  
第2个矩形框的坐标位置是：(286, 212), 周长是: 170, 面积是: 1764  
第3个矩形框的坐标位置是：(483, 163), 周长是: 236, 面积是: 3040  
经统计, 共3个目标

第1个矩形框的坐标位置是：(588, 239), 周长是: 220, 面积是: 2064  
第2个矩形框的坐标位置是：(295, 212), 周长是: 254, 面积是: 3526  
第

第1个矩形框的坐标位置是：(730, 326), 周长是: 206, 面积是: 2470  
第2个矩形框的坐标位置是：(746, 280), 周长是: 134, 面积是: 990  
第3个矩形框的坐标位置是：(557, 224), 周长是: 366, 面积是: 8030  
第4个矩形框的坐标位置是：(381, 219), 周长是: 138, 面积是: 1118  
第5个矩形框的坐标位置是：(434, 190), 周长是: 246, 面积是: 3402  
经统计, 共5个目标

第1个矩形框的坐标位置是：(730, 278), 周长是: 320, 面积是: 4636  
第2个矩形框的坐标位置是：(556, 225), 周长是: 350, 面积是: 7276  
第3个矩形框的坐标位置是：(382, 219), 周长是: 190, 面积是: 2256  
第4个矩形框的坐标位置是：(434, 188), 周长是: 246, 面积是: 3440  
经统计, 共4个目标

第1个矩形框的坐标位置是：(720, 272), 周长是: 314, 面积是: 4680  
第2个矩形框的坐标位置是：(545, 224), 周长是: 358, 面积是: 7828  
第3个矩形框的坐标位置是：(383, 217), 周长是: 144, 面积是: 1292  
第4个矩形框的坐标位置是：(435, 187), 周长是: 242, 面积是: 3318  
经统计, 共4个目标

第1个矩形框的坐标位置是：(712, 270), 周长是: 324, 面积是: 5117  
第2个矩形框的坐标位置是：(537, 224), 周长是: 356, 面积是: 7821  
第3个矩形框的坐标位置是：(381, 216), 周长是: 114, 面积是: 722  
第4个矩形框的坐标位置是：(438, 185), 周长是: 234, 面积是: 2960  
经统计, 共4个目标

第1个矩形框的坐标位置是：(707, 269), 周长是: 324, 面积是: 5040  
第2个矩形框的坐标位置是：(532, 221), 周长是: 376, 面积是: 8772  
第3个矩形框的坐标位置是：(379, 215), 周长是: 118, 面积是: 780  
第4个矩形框的坐标位置是：(4

第1个矩形框的坐标位置是：(486, 190), 周长是: 162, 面积是: 1368  
第2个矩形框的坐标位置是：(207, 180), 周长是: 376, 面积是: 8692  
第3个矩形框的坐标位置是：(560, 179), 周长是: 268, 面积是: 3760  
第4个矩形框的坐标位置是：(484, 168), 周长是: 102, 面积是: 608  
第5个矩形框的坐标位置是：(449, 165), 周长是: 232, 面积是: 2964  
第6个矩形框的坐标位置是：(256, 52), 周长是: 114, 面积是: 770  
经统计, 共6个目标

第1个矩形框的坐标位置是：(487, 193), 周长是: 148, 面积是: 1173  
第2个矩形框的坐标位置是：(207, 182), 周长是: 392, 面积是: 9348  
第3个矩形框的坐标位置是：(561, 176), 周长是: 268, 面积是: 3648  
第4个矩形框的坐标位置是：(485, 167), 周长是: 94, 面积是: 522  
第5个矩形框的坐标位置是：(450, 166), 周长是: 228, 面积是: 2808  
第6个矩形框的坐标位置是：(256, 54), 周长是: 112, 面积是: 748  
经统计, 共6个目标

第1个矩形框的坐标位置是：(487, 193), 周长是: 146, 面积是: 1150  
第2个矩形框的坐标位置是：(207, 182), 周长是: 398, 面积是: 9660  
第3个矩形框的坐标位置是：(563, 176), 周长是: 256, 面积是: 3255  
第4个矩形框的坐标位置是：(486, 165), 周长是: 94, 面积是: 522  
第5个矩形框的坐标位置是：(450, 165), 周长是: 228, 面积是: 2808  
第6个矩形框的坐标位置是：(255, 54), 周长是: 114, 面积是: 782  
经统计, 共6个目标

第1个矩形框的坐标位置是：(207, 184), 周长是: 406, 面积是: 10062  
第2个矩形框的坐标位置是：(562, 172), 周长是: 260, 面积是: 3384  
第3个矩形框的坐标位置是：(451, 160), 周长是: 284,

第1个矩形框的坐标位置是：(712, 220), 周长是: 278, 面积是: 4278  
第2个矩形框的坐标位置是：(483, 210), 周长是: 234, 面积是: 2720  
第3个矩形框的坐标位置是：(626, 131), 周长是: 240, 面积是: 3311  
第4个矩形框的坐标位置是：(549, 130), 周长是: 270, 面积是: 4484  
第5个矩形框的坐标位置是：(173, 68), 周长是: 120, 面积是: 836  
经统计, 共5个目标

第1个矩形框的坐标位置是：(701, 216), 周长是: 294, 面积是: 4850  
第2个矩形框的坐标位置是：(492, 211), 周长是: 234, 面积是: 2720  
第3个矩形框的坐标位置是：(629, 130), 周长是: 234, 面积是: 3080  
第4个矩形框的坐标位置是：(546, 130), 周长是: 286, 面积是: 5100  
第5个矩形框的坐标位置是：(171, 69), 周长是: 114, 面积是: 740  
经统计, 共5个目标

第1个矩形框的坐标位置是：(698, 218), 周长是: 282, 面积是: 4464  
第2个矩形框的坐标位置是：(494, 213), 周长是: 252, 面积是: 3528  
第3个矩形框的坐标位置是：(548, 130), 周长是: 294, 面积是: 5402  
第4个矩形框的坐标位置是：(636, 129), 周长是: 228, 面积是: 2808  
第5个矩形框的坐标位置是：(168, 69), 周长是: 128, 面积是: 988  
经统计, 共5个目标

第1个矩形框的坐标位置是：(385, 225), 周长是: 110, 面积是: 700  
第2个矩形框的坐标位置是：(688, 216), 周长是: 260, 面积是: 3600  
第3个矩形框的坐标位置是：(508, 214), 周长是: 238, 面积是: 2940  
第4个矩形框的坐标位置是：(639, 129), 周长是: 230, 面积是: 2926  
第5个矩形框的坐标位置是：(558, 128), 周长是: 276, 面积是: 4745  
第6个矩形框的坐标位置是：(162, 70), 周长是: 12

第1个矩形框的坐标位置是：(631, 235), 周长是: 262, 面积是: 3478  
第2个矩形框的坐标位置是：(722, 229), 周长是: 240, 面积是: 2871  
第3个矩形框的坐标位置是：(432, 227), 周长是: 116, 面积是: 697  
第4个矩形框的坐标位置是：(385, 223), 周长是: 150, 面积是: 1224  
第5个矩形框的坐标位置是：(482, 185), 周长是: 252, 面积是: 3569  
第6个矩形框的坐标位置是：(430, 185), 周长是: 130, 面积是: 966  
第7个矩形框的坐标位置是：(708, 110), 周长是: 186, 面积是: 1656  
第8个矩形框的坐标位置是：(622, 110), 周长是: 230, 面积是: 3234  
第9个矩形框的坐标位置是：(90, 85), 周长是: 132, 面积是: 1025  
经统计, 共9个目标

第1个矩形框的坐标位置是：(620, 233), 周长是: 256, 面积是: 3312  
第2个矩形框的坐标位置是：(728, 232), 周长是: 250, 面积是: 3400  
第3个矩形框的坐标位置是：(432, 230), 周长是: 128, 面积是: 975  
第4个矩形框的坐标位置是：(384, 223), 周长是: 158, 面积是: 1428  
第5个矩形框的坐标位置是：(431, 187), 周长是: 126, 面积是: 882  
第6个矩形框的坐标位置是：(472, 181), 周长是: 240, 面积是: 2871  
第7个矩形框的坐标位置是：(626, 110), 周长是: 234, 面积是: 3392  
第8个矩形框的坐标位置是：(709, 109), 周长是: 180, 面积是: 1449  
第9个矩形框的坐标位置是：(85, 87), 周长是: 126, 面积是: 920  
经统计, 共9个目标

第1个矩形框的坐标位置是：(732, 233), 周长是: 240, 面积是: 3024  
第2个矩形框的坐标位置是：(616, 231), 周长是: 244, 面积是: 2760  
第3个矩形框的坐标位置是：(385, 223), 周长是: 170, 面积是: 1674 

...

第1个矩形框的坐标位置是：(120, 320), 周长是: 392, 面积是: 8515  
第2个矩形框的坐标位置是：(432, 244), 周长是: 246, 面积是: 2726  
第3个矩形框的坐标位置是：(400, 244), 周长是: 246, 面积是: 2852  
第4个矩形框的坐标位置是：(279, 162), 周长是: 278, 面积是: 4648  
第5个矩形框的坐标位置是：(164, 157), 周长是: 204, 面积是: 2025  
第6个矩形框的坐标位置是：(490, 155), 周长是: 280, 面积是: 4576  
第7个矩形框的坐标位置是：(397, 147), 周长是: 190, 面积是: 1836  
经统计, 共7个目标

第1个矩形框的坐标位置是：(120, 320), 周长是: 384, 面积是: 8255  
第2个矩形框的坐标位置是：(434, 303), 周长是: 116, 面积是: 805  
第3个矩形框的坐标位置是：(432, 253), 周长是: 118, 面积是: 528  
第4个矩形框的坐标位置是：(391, 243), 周长是: 266, 面积是: 3720  
第5个矩形框的坐标位置是：(292, 160), 周长是: 258, 面积是: 3740  
第6个矩形框的坐标位置是：(157, 158), 周长是: 222, 面积是: 2660  
第7个矩形框的坐标位置是：(492, 155), 周长是: 268, 面积是: 4128  
第8个矩形框的坐标位置是：(403, 146), 周长是: 182, 面积是: 1564  
经统计, 共8个目标

第1个矩形框的坐标位置是：(117, 318), 周长是: 376, 面积是: 7611  
第2个矩形框的坐标位置是：(434, 303), 周长是: 108, 面积是: 704  
第3个矩形框的坐标位置是：(390, 234), 周长是: 288, 面积是: 4223  
第4个矩形框的坐标位置是：(294, 160), 周长是: 254, 面积是: 3526  
第5个矩形框的坐标位置是：(152, 160), 周长是: 226, 面积是: 2886  
第6个矩形框的坐标位置是：(482, 154), 周长是: 278, 面积是:

第1个矩形框的坐标位置是：(66, 268), 周长是: 320, 面积是: 5724  
第2个矩形框的坐标位置是：(416, 242), 周长是: 82, 面积是: 364  
第3个矩形框的坐标位置是：(603, 209), 周长是: 310, 面积是: 5796  
第4个矩形框的坐标位置是：(200, 172), 周长是: 358, 面积是: 7668  
第5个矩形框的坐标位置是：(354, 156), 周长是: 238, 面积是: 3300  
第6个矩形框的坐标位置是：(495, 130), 周长是: 340, 面积是: 5376  
经统计, 共6个目标

第1个矩形框的坐标位置是：(65, 265), 周长是: 332, 面积是: 6160  
第2个矩形框的坐标位置是：(600, 208), 周长是: 298, 面积是: 5208  
第3个矩形框的坐标位置是：(490, 178), 周长是: 248, 面积是: 3520  
第4个矩形框的坐标位置是：(198, 173), 周长是: 344, 面积是: 7035  
第5个矩形框的坐标位置是：(349, 156), 周长是: 248, 面积是: 3675  
第6个矩形框的坐标位置是：(514, 129), 周长是: 188, 面积是: 1848  
经统计, 共6个目标

第1个矩形框的坐标位置是：(59, 259), 周长是: 318, 面积是: 5450  
第2个矩形框的坐标位置是：(432, 242), 周长是: 132, 面积是: 1080  
第3个矩形框的坐标位置是：(590, 208), 周长是: 292, 面积是: 4968  
第4个矩形框的坐标位置是：(490, 178), 周长是: 244, 面积是: 3360  
第5个矩形框的坐标位置是：(192, 175), 周长是: 364, 面积是: 8181  
第6个矩形框的坐标位置是：(331, 159), 周长是: 258, 面积是: 4028  
第7个矩形框的坐标位置是：(521, 129), 周长是: 184, 面积是: 1716  
经统计, 共7个目标

第1个矩形框的坐标位置是：(50, 259), 周长是: 336, 面积是: 6527  
第2个矩形框的坐标位置是：(433, 242), 周长是: 

第1个矩形框的坐标位置是：(121, 216), 周长是: 270, 面积是: 4136  
第2个矩形框的坐标位置是：(332, 202), 周长是: 274, 面积是: 4312  
第3个矩形框的坐标位置是：(269, 192), 周长是: 310, 面积是: 5916  
第4个矩形框的坐标位置是：(445, 187), 周长是: 254, 面积是: 3570  
第5个矩形框的坐标位置是：(555, 184), 周长是: 386, 面积是: 7752  
第6个矩形框的坐标位置是：(8, 170), 周长是: 284, 面积是: 5040  
第7个矩形框的坐标位置是：(620, 115), 周长是: 172, 面积是: 1560  
第8个矩形框的坐标位置是：(739, 56), 周长是: 122, 面积是: 798  
经统计, 共8个目标

第1个矩形框的坐标位置是：(120, 216), 周长是: 268, 面积是: 4128  
第2个矩形框的坐标位置是：(336, 202), 周长是: 276, 面积是: 4361  
第3个矩形框的坐标位置是：(264, 197), 周长是: 320, 面积是: 6351  
第4个矩形框的坐标位置是：(442, 185), 周长是: 234, 面积是: 2610  
第5个矩形框的坐标位置是：(552, 183), 周长是: 404, 面积是: 8832  
第6个矩形框的坐标位置是：(12, 170), 周长是: 262, 面积是: 4260  
第7个矩形框的坐标位置是：(615, 114), 周长是: 206, 面积是: 2542  
第8个矩形框的坐标位置是：(739, 58), 周长是: 122, 面积是: 840  
经统计, 共8个目标

第1个矩形框的坐标位置是：(125, 216), 周长是: 276, 面积是: 4437  
第2个矩形框的坐标位置是：(336, 203), 周长是: 282, 面积是: 4550  
第3个矩形框的坐标位置是：(261, 200), 周长是: 288, 面积是: 4988  
第4个矩形框的坐标位置是：(438, 186), 周长是: 246, 面积是: 3182  
第5个矩形框的坐标位置是：(547, 183), 周长是: 414, 面积是: 952

第1个矩形框的坐标位置是：(646, 201), 周长是: 262, 面积是: 3100  
第2个矩形框的坐标位置是：(440, 183), 周长是: 402, 面积是: 8768  
第3个矩形框的坐标位置是：(234, 176), 周长是: 514, 面积是: 15810  
第4个矩形框的坐标位置是：(673, 122), 周长是: 202, 面积是: 2418  
第5个矩形框的坐标位置是：(732, 53), 周长是: 166, 面积是: 1692  
经统计, 共5个目标

第1个矩形框的坐标位置是：(457, 224), 周长是: 312, 面积是: 5459  
第2个矩形框的坐标位置是：(644, 199), 周长是: 262, 面积是: 3100  
第3个矩形框的坐标位置是：(433, 184), 周长是: 230, 面积是: 3034  
第4个矩形框的坐标位置是：(234, 176), 周长是: 514, 面积是: 15700  
第5个矩形框的坐标位置是：(673, 122), 周长是: 206, 面积是: 2496  
第6个矩形框的坐标位置是：(731, 52), 周长是: 170, 面积是: 1776  
经统计, 共6个目标

第1个矩形框的坐标位置是：(466, 228), 周长是: 306, 面积是: 5300  
第2个矩形框的坐标位置是：(631, 199), 周长是: 282, 面积是: 4268  
第3个矩形框的坐标位置是：(431, 185), 周长是: 212, 面积是: 2409  
第4个矩形框的坐标位置是：(239, 176), 周长是: 490, 面积是: 13600  
第5个矩形框的坐标位置是：(673, 125), 周长是: 206, 面积是: 2470  
第6个矩形框的坐标位置是：(732, 52), 周长是: 166, 面积是: 1692  
经统计, 共6个目标

第1个矩形框的坐标位置是：(470, 229), 周长是: 298, 面积是: 4900  
第2个矩形框的坐标位置是：(624, 200), 周长是: 294, 面积是: 4896  
第3个矩形框的坐标位置是：(255, 176), 周长是: 458, 面积是: 11040  
第4个矩形框的坐标位置是：(676, 125)

第1个矩形框的坐标位置是：(432, 271), 周长是: 268, 面积是: 2808  
第2个矩形框的坐标位置是：(591, 225), 周长是: 458, 面积是: 11988  
第3个矩形框的坐标位置是：(378, 162), 周长是: 214, 面积是: 2400  
第4个矩形框的坐标位置是：(215, 160), 周长是: 360, 面积是: 7371  
第5个矩形框的坐标位置是：(647, 144), 周长是: 214, 面积是: 2520  
第6个矩形框的坐标位置是：(732, 48), 周长是: 170, 面积是: 1764  
经统计, 共6个目标

第1个矩形框的坐标位置是：(432, 272), 周长是: 304, 面积是: 4687  
第2个矩形框的坐标位置是：(594, 228), 周长是: 452, 面积是: 11680  
第3个矩形框的坐标位置是：(384, 161), 周长是: 212, 面积是: 2280  
第4个矩形框的坐标位置是：(215, 160), 周长是: 356, 面积是: 7192  
第5个矩形框的坐标位置是：(648, 145), 周长是: 208, 面积是: 2380  
第6个矩形框的坐标位置是：(732, 48), 周长是: 170, 面积是: 1764  
经统计, 共6个目标

第1个矩形框的坐标位置是：(435, 274), 周长是: 316, 面积是: 5341  
第2个矩形框的坐标位置是：(593, 232), 周长是: 456, 面积是: 12035  
第3个矩形框的坐标位置是：(385, 161), 周长是: 216, 面积是: 2475  
第4个矩形框的坐标位置是：(215, 158), 周长是: 352, 面积是: 6903  
第5个矩形框的坐标位置是：(648, 145), 周长是: 214, 面积是: 2556  
第6个矩形框的坐标位置是：(732, 48), 周长是: 170, 面积是: 1764  
经统计, 共6个目标

第1个矩形框的坐标位置是：(437, 275), 周长是: 320, 面积是: 5559  
第2个矩形框的坐标位置是：(602, 232), 周长是: 454, 面积是: 11760  
第3个矩形框的坐标位置是：(386, 161)

**运行结果**：此处只展示其中的某一帧的截图

<img align="left" width=500 src="./screenshot/4.jpg" />

### 三、安装ImageWatch，并在代码中通过设置断点，观察处理中间结果图像

说明：ImageWatch是基于VS IDE的一款扩展插件，可以在代码中显示运行的过程图像等调试信息

<img width="700" src="./screenshot/2.png" />

其他vs插件补充：

- VS Color Output 扩展插件 显示区分不同的颜色，方便调试
- VS Line Counter 扩展插件 统计项目所有文件代码行数
- Super Charger 扩展插件 ：代码流线、超级拷贝粘贴(栈)、超级Find(颜色区分)、智能注释等等

### 四、使用光流估计方法，在前述测试视频上计算特征点，进一步进行特征点光流估计

In [1]:
import cv2 as cv
import numpy as np

videoFileName = './video/vtest.avi'

# 角点检测参数
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)

# lucas kanade 光流法参数
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.032)
)

cap = cv.VideoCapture(videoFileName)

# 计算第一帧特征点
ret, prev = cap.read()
# OpenCV只支持灰度图像的特征检测，转换为灰度图像
prevGray = cv.cvtColor(prev, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(prevGray, mask=None, **feature_params)

# 特征点集
points = np.zeros_like(prev)
while True:
    ret, frame = cap.read()
    # 没读到当前帧，结束
    if not ret:
        break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # 计算光流 参数：prevGray是上一帧灰度图像，gray是当前帧灰度图像，最后一个**lk_params是前面指定的检测参数
    # 返回值p1表示检测到所有光流点的列表，st描述了每一个特征点计算光流时候的置信度
    p1, st, err = cv.calcOpticalFlowPyrLK(prevGray, gray, p0, None, **lk_params)

    # 选取好的跟踪点 只选择置信度好的光流检测结果作为输出
    goodPoints = p1[st == 1]
    goodPrevPoints = p0[st == 1]

    # 在结果图像中叠加画出特征点和计算出来的光流向量
    res = frame.copy()
    drawColor1 = (0, 0, 255)  # 红色
    drawColor2 = (0, 255, 0)  # 绿色

    for i, (cur, prev) in enumerate(zip(goodPoints, goodPrevPoints)):
        # 得到上一帧对应特征点的位置
        x0, y0 = cur.ravel()
        # 得到这一帧对应特征点的位置
        x1, y1 = prev.ravel()
        cv.line(res, (x0, y0), (x1, y1), drawColor1) # 绘制向量
        cv.line(points, (x0, y0), (x1, y1), drawColor1) # 绘制线条
        frame = cv.circle(res, (x0, y0), 3, drawColor2) # 绘制圆点

    trace = cv.add(frame, points) #
    # 更新上一帧，在下一阵的时候，当前帧变成了上一帧
    prevGray = gray.copy()
    # 同样更新上一帧对应的特征点
    p0 = goodPoints.reshape(-1, 1, 2)

    # 显示计算结果图像
    # cv.imshow('Detection Result', res)
    cv.imshow('Trace', trace)
    # print('当前较好的特征点个数: ' + str(count))

    key = cv.waitKey(30)  # 每一帧间隔30ms
    # 按下ESC，退出
    if key == 27:
        break

cap.release()
cv.destroyAllWindows()

**相关说明**：此处只展示其中的某一帧的截图, 光流估计检测光流点的时候依靠的是角点，并不能准确检测到所有人，一些车窗和物体上被检测到了光流点, 光流计算需要寻找图像中的特征点，比如深色衣服，没有明显的灰度变化，很难被选举成特征，这也是光流估计本身的一些缺点, 在实际中光流估计是一类很重要的算法，如果把它用在目标的运动估计，通常要和其他方法结合使用以得到最好的结果

<img align="left" width=500 src="./screenshot/5.jpg" /> <br />